In [13]:
import dask
from fake_useragent import UserAgent
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import pandas as pd
import json
import time
import bookquery
import os
import io
import json
from tqdm import tqdm
from os.path import join,isfile
from os import listdir
from collections import Counter
import dask
import numpy as np
from pymorphy2 import MorphAnalyzer

ua = UserAgent()

In [14]:
def execute_query(qquery, entity_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(entity_id, f = 0):
    res = []
    results= execute_query(bookquery.entity_finding, entity_id)
    for result in results:
        entities = {}
        entities['itemid'] = result['item']['value'].split('/')[-1]
        entities['label'] = result['itemLabel']['value']
        res.append(entities)
    return pd.DataFrame(data = res, columns = ['itemid', 'label'])
        

In [15]:
ent = entities_sparqul("Q119253")
ent

,itemid,label
0,Q119196,Кермадек
1,Q1423293,Yap Trench
2,Q3079494,Beaufort's Dyke
3,Q3373484,Hikurangi Trench
4,Q5434800,Farallon Trench
5,Q118873,Перуанско-Чилийский жёлоб
6,Q164040,Центральноамериканский жёлоб
7,Q9325582,Timor Trough
8,Q56319989,Q56319989
9,Q118537,Японский жёлоб


In [16]:
def execute_query_2(qquery, entity_id , pred_id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id , pred_id =pred_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

# def predicates(entity_id,pred_id):
#     res = []
#     results= execute_query(bookquery.entity_finding, entity_id)
#     for result in results:
#         entities = {}
#         entities['item'] = result['item']['value'].split('/')[-1]
#         entities['label'] = result['itemLabel']['value']
#         entities['answer'] = result['answer']['value']
#         res.append(entities)
#     return pd.DataFrame(data = res, columns = ['item', 'label' , 'answer'])


def predicates(entity_id, pred_id, label, f = 0):
        res = []
        if f > 5:
            return None
        try:
            results= execute_query_2(bookquery.predicat_reciver, entity_id , pred_id)
            if len(results)>1:
                results = [results[0]]
            entities = {}
            for result in results:
                entities['itemid'] = result['item']['value'].split('/')[-1]
                entities['label'] = result['itemLabel']['value']
                entities['answer'] = result['answer']['value']
                entities['point'] = 1
                res.append(entities)
            if len(res) == 0:
                entities['itemid'] = entity_id
                entities['label'] = label
                entities['answer'] = None
                entities['point'] = 0
                res.append(entities)
            return pd.DataFrame(data = res, columns = ['itemid', 'label' , 'answer' , "point"])
        except Exception as e:
#             print(e)
            time.sleep(5)
#             print('Try again ', entity_id, pred_id, f)
            return predicates(entity_id,pred_id, label ,f + 1)

In [5]:
# def predicates(entity_id, pred_id, label, f = 0):
#     res = []
#     results= execute_query_2(bookquery.predicat_reciver, entity_id , pred_id)
#     if len(results)>1:
#         results = [results[0]]
#     entities = {}
#     for result in results:
#         print(result)
#         entities['itemid'] = result['item']['value'].split('/')[-1]
#         entities['label'] = result['itemLabel']['value']
#         entities['answer'] = result['answer']['value']
#         entities['point'] = 1
#         res.append(entities)
#     if len(res) == 0:
#         entities['itemid'] = entity_id
#         entities['label'] = label
#         entities['answer'] = None
#         entities['point'] = 0
#         res.append(entities)
#     return pd.DataFrame(data = res, columns = ['itemid', 'label' , 'answer' , "point"])

In [17]:
def answers(entity_id, pred_id):
    pred_mas
    res = []
    df_items = entities_sparqul(entity_id)
    for k,v in df_items.iterrows():
        res.append(dask.delayed(predicates)(v.itemid , pred_id , v.label))
    result = dask.compute(*res)
    final_result = pd.concat(result)
    final_result
    return final_result

In [7]:
df = answers("Q149621" , "P625")
summa = round(df.point.sum()/len(df),2) * 100
if len(df)<50:
    print(f'Процент ответов по всем значениям:{summa} %')
else:
    print(f'Процент ответов по выборке:{summa} %')
df

NameError: name 'pred_mas' is not defined

In [ ]:
df = pd.DataFrame()

In [ ]:
def only_rus(ent):
    res = []
    d = {'itemid' : [] , 'label' : []}
    morph = MorphAnalyzer()
    for j in ent['label'].tolist():
        tag = morph.parse(j)[0].tag
        if not 'LATN' in tag:
            d['itemid'].append(ent[ent['label'] == j]['itemid'].values[0])
            d['label'].append(j)
            res = pd.DataFrame(data = d, columns = ['itemid', 'label'])
    return res

def answers_height(entity_id):
    pred_mas = ['P2660' , 'P2044' , 'P2048']
    glob_mas = []
    for item in entity_id:
        df_items = only_rus(entities_sparqul(item))
#         df_items =entities_sparqul(item)
        if len(df_items) == 0:
            continue
        print(f'Количество представителей : {len(df_items)}')
        entities = {}
        entities['label'] = item
        for predicate in pred_mas:
            res = []
#             print(df_items)
            for k,v in df_items.iterrows():
                res.append(dask.delayed(predicates)(v.itemid , predicate , v.label))
            result = dask.compute(*res)
            final_result = pd.concat(result)
            summa = round(final_result.point.sum()/len(final_result),2) * 100
            entities[predicate] = summa
        glob_mas.append(entities)
    return pd.DataFrame(data = glob_mas)



In [ ]:
%%time
mass= ['Q15921555'
]
statistics = answers_height(mass)

In [ ]:
statistics

In [ ]:
statistics.mean()

In [ ]:
df = pd.read_excel(r'csv\Height.xlsx')
df = pd.concat([ df,statistics])
df.to_excel(r'csv\Height.xlsx',sheet_name="Высота", encoding='utf-8-sig' , index = False)

In [18]:
def only_rus(ent):
    res = []
    d = {'itemid' : [] , 'label' : []}
    morph = MorphAnalyzer()
    for j in ent['label'].tolist():
        tag = morph.parse(j)[0].tag
        if not 'LATN' in tag:
            d['itemid'].append(ent[ent['label'] == j]['itemid'].values[0])
            d['label'].append(j)
            res = pd.DataFrame(data = d, columns = ['itemid', 'label'])
    return res

def answers_location(entity_id,syn_class):
    pred_mas = ['P206' , 'P706' , 'P131' , 'P276' , 'P17' , 'P30' , "P361" , 'P669']
    glob_mas = []
    for item in entity_id:
        df_items = only_rus(entities_sparqul(item))
#         df_items =entities_sparqul(item)
        if len(df_items) == 0:
            continue
        print(f'Количество представителей у {syn_class} : {len(df_items)}')
        entities = {}
        entities['class'] = syn_class
        entities['label'] = item 
        for predicate in pred_mas:
            res = []
#             print(df_items)
            for k,v in df_items.iterrows():
                res.append(dask.delayed(predicates)(v.itemid , predicate , v.label))
            result = dask.compute(*res)
            final_result = pd.concat(result)
            summa = round(final_result.point.sum()/len(final_result),2) * 100
            entities[predicate] = summa
        glob_mas.append(entities)
    df = pd.DataFrame(data = glob_mas)
    if len(df) == 0:
            return None
    df.columns = ['Class' , 'QID' , 'На берегу водоема' , 'В пределах прир.-геогр. об' , 'Админ.-террит. ед.' , 'Местонахождение',
                      'Страна',"Часть света" , "Часть от" , "Расположен на улице"]
    return df



In [11]:
data =  { "dr:metro_station" : ["Q928830"],
         "dr:cliff" : ["Q107679"],
         "dr:tree" : ["Q10884"],
         "dr:mineral_deposit" : ["Q15104915"],
         "dr:oasis" : ["Q43742"],
         "dr:moat" : ["Q88480"],
         "dr:slope" : ["Q944588"],
         "dr:underground_mine" : ["Q4232315"],
         "dr:artesian_well" : ["Q707912"],
         "dr:river_delta" : ["Q43197"],
         "dr:railway_station" : ["Q55488"],
         "dr:sinkhole" : ["Q188734"],
         "dr:kurgan" : ["Q5737"],
         "dr:spa_town" : ["Q4946461"],
         "dr:tombstone" : ["Q203443"],
         "dr:dam" : ["Q12323"],
         "dr:prairie" : ["Q194281"],
         "dr:mine" : ["Q820477"],
         "dr:art_of_sculpture" : ["Q11634"],
         "dr:spire" : ["Q852590"],
         "dr:unesco_world_heritage_site" : ["Q9259"],
         "dr:wonders_of_the_world" : ["Q209287"],
         "dr:brook" : ["Q63565252"],
         "dr:hot_spring" : ["Q177380"],
         "dr:group_of_lakes" : ["Q5926864"],
         "dr:continental_area_and_surrounding_islands" : ["Q2418896"],
         "dr:time_zone" : ["Q12143"],
         "dr:meridian" : ["Q32099"],
         "dr:track" : ["Q160342"],
         "dr:tundra" : ["Q43262"],
         "dr:continental_shelf" : ["Q134851"],
         "dr:county" : ["Q28575"],
         "dr:governorate" : ["Q86622"],
         "dr:department_of_france" : ["Q6465"],
         "dr:province_of_thailand" : ["Q50198"],
         "dr:dr:province" : ["Q34876"],
         "dr:quarter" : ["Q2983893"],
         "dr:economic_region" : ["Q1973901"],
         "dr:continent" : ["Q5107"],
         "dr:coast" : ["Q93352"],
         "dr:drainage_basin" : ["Q166620"] 
}

In [20]:
data = {"dr:railway":["Q22667"],
        "dr:body_of_water": ["Q15324"],
        "dr:watercourse": ["Q355304"],
        "dr:rapid_transit" :["Q5503"],
        "dr:coastal_line": ["Q1701967"],
        "dr:steppe": ["Q123991"]}

In [21]:
%%time
for k,v in tqdm(data.items()):
    statistics = answers_location(v,k)
    df = pd.read_excel(r'csv\Location.xlsx')
    df = pd.concat([ df,statistics])
    df.to_excel(r'csv\Location.xlsx',sheet_name="Расположение", encoding='utf-8-sig' , index = False)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

Количество представителей у dr:railway : 48


 17%|██████████████                                                                      | 1/6 [00:22<01:50, 22.10s/it]

Количество представителей у dr:body_of_water : 34


 33%|████████████████████████████                                                        | 2/6 [00:37<01:20, 20.00s/it]

...

Количество представителей у dr:rapid_transit : 204


 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [02:16<00:24, 24.51s/it]

Количество представителей у dr:steppe : 25


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [02:27<00:00, 24.65s/it]


Wall time: 2min 27s


In [19]:
%%time
mass= ['Q123991']
statistics = answers_location(mass,'dr:steppe')

Wall time: 715 ms


In [ ]:
statistics

In [ ]:
statistics.mean()

In [ ]:
        "dr:posyolok":["Q2514025"],
        "dr:street":["Q79007"]
        "dr:country":["Q6256"],
         "dr:republic(базовый класс)" : ["Q7270"],
         "dr:republic_of_russia" : ["Q41162"],
         "dr:federated_state" : ["Q107390"],
         "dr:oblast" : ["Q171308"],
         "dr:district" : ["Q149621"],
         "dr:region" : ["Q82794"],
         "dr:region_of_russia_subject" : ["Q43263"],
         "dr:region_of_russia_economic_region" : ["Q565410"],
         "dr:still_waters" : ["Q337567"],
         "dr:ocean" : ["Q943"],
         "dr:pond" : ["Q3253281"],
         "dr:reservoir" : ["Q131681"],
         "dr:march" : ["Q30198"],
         "dr:lake" : ["Q23397"],
         "dr:canal" : ["Q12284"],
         "dr:bay" : ["Q39594"],
         "dr:sea" : ["Q165"],
         "dr:river" : ["Q4022"],
         "dr:architectural_structure" : ["Q811979",'Q12518','Q179700','Q4989906','Q1068623','Q39715','Q32815','Q33506','Q174782',
                                         'Q1501','Q43483','Q483110','Q1281105','Q381885','Q13409250','Q202570','Q863454','Q483453'],
         "dr:building" : ['Q41176','Q11303','Q3947','Q23413','Q16970','Q2977','Q24354','Q1248784','Q12292478','Q19364326',
                          'Q671224','Q488444','Q55713852','Q1412364'],
         "dr:pyramid" : ["Q12516"],
         "dr:bridge" : ["Q12280"],
         "dr:tunnel" : ["Q44377"],
         "dr:road" : ["Q34442"],
         "dr:port" : ["Q44782"],
         "dr:waterfall" : ["Q34038"],
         "dr:desert" : ["Q8514"],
         "dr:peninsula" : ["Q34763"],
         "dr:strait" : ["Q37901"],
         "dr:cave" : ["Q35509"],
         "dr:archipelago" : ["Q33837"],
         "dr:glacier" : ["Q35666"],
         "dr:nature_reserve" : ["Q179049"],
         "dr:geyser" : ["Q83471"],
         "dr:cape" : ["Q185113"],
         "dr:mountain_pass" : ["Q133056"],
         "dr:beach" : ["Q40080"],
         "dr:cove" : ["Q31615"],
         "dr:quarry" : ["Q188040"],
         "dr:isthmus" : ["Q93267"],
         "dr:fault" : ["Q47089"],
         "dr:borehole" : ["Q502102"],